In [20]:
def new_slurm_script(script_name, cpus_per_task='1', mem_per_cpu='5G', array_t='1-9000', array_tc='250'):



    
    with open(script_name, 'w') as f:
        f.write('''#! /bin/bash
        
#SBATCH --job-name={0}
#SBATCH --output={0}.out
#SBATCH --error={0}.error
#SBATCH --cpus-per-task={1}
#SBATCH --mem-per-cpu={2}
#SBATCH --array={3}%{4}
'''.format(script_name, cpus_per_task, mem_per_cpu, array_t, array_tc))


def map_slurm_task_id(script_name, file_list, var_name):

    # generate file list with ' '
    file_list_str = ' '.join(file_list)


    with open(script_name, 'a') as f:
        f.write('''
{0}s=({1})
{0}=${{{0}s[$SLURM_ARRAY_TASK_ID -1]}}
'''.format(var_name, file_list_str))

def unzip_all(script_name, var_name):
    with open(script_name, 'a') as f:
        f.write('''
gunzip ${0}'''.format(var_name))
def star_cmd(script_name, genome, fastq_var, output_path):
    with open(script_name, 'a') as f:
        f.write('''
STAR --genomeDir {0} \
--runThreadN 8 \
--readFilesIn {1} \
--outFileNamePrefix {2}/$ID \
--outSAMtype BAM SortedByCoordinate \
--outSAMunmapped Within \
--outSAMattributes Standard \
--outTmpDir ./_STARtmp$ID \


'''.format(genome, fastq_var, output_path))
def feature_count(gtf, outfile, infile_list, script_name):
    with open(script_name, 'a') as f:
        f.write('''
featureCounts -t exon -g gene_id -a {0} -o {1} {2}
'''.format(gtf, outfile, ' '.join(infile_list)))

In [2]:
import os
root_dir = '/cellar/users/hsher/Data/ideker/sc/'
human_genome_index = '/nrnb/users/hsher/star_hg38'
mouse_genome_index = '/nrnb/users/hsher/star_mous'

In [3]:
import pandas as pd
rnaseq = pd.read_excel('RNA-seq.xlsx')

In [4]:
rnaseq

,Unnamed: 0,GEO accession,species,SRA
0,Darmanis,GSE67835,human,SRP057196
1,Zhang,GSE73721,human,SRP064454
2,Zhang,GSE52564,mouse,SRP033200
3,Zeisel,GSE60361,mouse,SRP045452
4,Tasic,GSE71585,mouse,SRP061902


In [5]:
number = 0

accession = rnaseq.loc[number, 'GEO accession']
srp_id = rnaseq.loc[number, 'SRA']
sps = rnaseq.loc[number, 'species']

fastq_path = [os.path.join(os.path.join(root_dir, srp_id), file) for file in os.listdir(os.path.join(root_dir, srp_id))]
prefix = list(set([srr.split('_')[0] for srr in fastq_path]))
srr_id = [p.split('/')[-1] for p in prefix]

sc_name = accession+'_star.run'
new_slurm_script(sc_name, '8', '8G', '1-{}'.format(len(srr_id)), '30')
map_slurm_task_id(sc_name, [p+ '_1.fastq' for p in prefix], 'FASTQ_ONE')
map_slurm_task_id(sc_name, [p+ '_2.fastq' for p in prefix], 'FASTQ_TWO')
map_slurm_task_id(sc_name, srr_id, 'ID')

if sps == 'human':
    star_cmd(sc_name, human_genome_index, '$FASTQ_ONE $FASTQ_TWO', '/nrnb/users/hsher/{}'.format(accession))

In [6]:
number = 1

accession = rnaseq.loc[number, 'GEO accession']
srp_id = rnaseq.loc[number, 'SRA']
sps = rnaseq.loc[number, 'species']

fastq_path = [os.path.join(os.path.join(root_dir, srp_id), file) for file in os.listdir(os.path.join(root_dir, srp_id))]
prefix = list(set([srr.split('_')[0] for srr in fastq_path]))
srr_id = [p.split('/')[-1] for p in prefix]

sc_name = accession+'_star.run'
new_slurm_script(sc_name, '8', '8G', '1-{}'.format(len(srr_id)), '30')
map_slurm_task_id(sc_name, [p+ '_1.fastq' for p in prefix], 'FASTQ_ONE')
map_slurm_task_id(sc_name, [p+ '_2.fastq' for p in prefix], 'FASTQ_TWO')
map_slurm_task_id(sc_name, srr_id, 'ID')

if sps == 'human':
    star_cmd(sc_name, human_genome_index, '$FASTQ_ONE $FASTQ_TWO', '/nrnb/users/hsher/{}'.format(accession))

In [7]:
number = 4

accession = rnaseq.loc[number, 'GEO accession']
srp_id = rnaseq.loc[number, 'SRA']
sps = rnaseq.loc[number, 'species']

fastq_path = [os.path.join(os.path.join(root_dir, srp_id), file) for file in os.listdir(os.path.join(root_dir, srp_id))]
prefix = list(set([srr.split('_')[0] for srr in fastq_path]))
srr_id = [p.split('/')[-1].replace('.fastq', '') for p in prefix]

sc_name = accession+'_star.run'
new_slurm_script(sc_name, '8', '8G', '1-{}'.format(len(srr_id)), '30')
map_slurm_task_id(sc_name, prefix, 'FASTQ_ONE')
map_slurm_task_id(sc_name, srr_id, 'ID')

if sps == 'human':
    star_cmd(sc_name, human_genome_index, '$FASTQ_ONE', '/nrnb/users/hsher/{}'.format(accession))
else:
    star_cmd(sc_name, mouse_genome_index, '$FASTQ_ONE', '/nrnb/users/hsher/{}'.format(accession))

In [8]:
number = 3

accession = rnaseq.loc[number, 'GEO accession']
srp_id = rnaseq.loc[number, 'SRA']
sps = rnaseq.loc[number, 'species']

fastq_path = [os.path.join(os.path.join(root_dir, srp_id), file) for file in os.listdir(os.path.join(root_dir, srp_id))]
prefix = list(set([srr.split('_')[0] for srr in fastq_path]))
srr_id = [p.split('/')[-1].replace('.fastq', '') for p in prefix]

sc_name = accession+'_star.run'
new_slurm_script(sc_name, '8', '8G', '1-{}'.format(len(srr_id)), '30')
map_slurm_task_id(sc_name, prefix, 'FASTQ_ONE')
map_slurm_task_id(sc_name, srr_id, 'ID')

if sps == 'human':
    star_cmd(sc_name, human_genome_index, '$FASTQ_ONE', '/nrnb/users/hsher/{}'.format(accession))
else:
    star_cmd(sc_name, mouse_genome_index, '$FASTQ_ONE', '/nrnb/users/hsher/{}'.format(accession))

In [25]:
bam_root = '/nrnb/users/hsher/'
gse = rnaseq['GEO accession']
sps = rnaseq['species']
out_path = bam_root + 'featureCount_out/'

In [26]:
i = 0
for g in gse:
    script_name = 'featureCount_{0}.run'.format(g)
    infile_list = [os.path.join(bam_root+g,file) for file in os.listdir(bam_root+g) if file.endswith(".bam")]
    outfile = out_path + g
    if sps[i] == 'human':
        gtf = '/cellar/users/hsher/Data/reference/Homo_sapiens.GRCh38.98.gtf'
    else:
        gtf = '/cellar/users/hsher/Data/star_mus_musculus_index/Mus_musculus.GRCm38.98.gtf'
    
    new_slurm_script(script_name, '8', '8G', '1-{}'.format(1), '30')
    feature_count(gtf, outfile, infile_list, script_name)
    
    